# Dr. Claude - Patient Care with AWS
## Personalized Medical Recommendations using Bedrock, DynamoDb, and SNS
### In this notebook we will:
- Create a vector store with OpenSearch Serverless
- Create a Knowledge Base with Amazon Bedrock
- Create a database for patient info with DynamoDB
- Query Anthropic's Claude 3 model for medical recommendations
- Create SNS topic and send the recommendations

### Prerequisites 
Add the following permissions to your SageMaker Domain role:
- AmazonBedrockFullAccess 
- AmazonDynamoDBFullAccess
- AmazonSNSFullAccess
- IAMFullAccess
- Add a custom [policy](./aoss_policy.json) for Amazon OpenSearch Serverless.


## Setup
Before running the rest of this notebook, you'll need to run the cells below to ensure necessary libraries are installed and connect to Bedrock.

In [ ]:
%pip install --upgrade pip --quiet
%pip install boto3 botocore langchain opensearch-py==2.3.1 --quiet

### Importing Libraries 
Import necessary libraries and perform initial setup required for subsequent AWS service interactions.

In [ ]:
import boto3
import botocore
from botocore.exceptions import ClientError
import json
import csv
from io import StringIO
from langchain_community.chat_models import BedrockChat
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever
import random
import pprint
import time
from policy_creator import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss
suffix = random.randrange(200, 900)

boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
sts_client = boto3.client('sts')
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
service = 'aoss'
account_id = sts_client.get_caller_identity()["Account"]
s3_suffix = f"{region_name}-{account_id}"

### Create S3 Bucket for the Knowledge Base Data
Create an S3 bucket which will be used to store data. If the bucket already exists, catch the error and notify the user.

In [ ]:
# Create an S3 client
s3 = boto3.client("s3")

# Create an S3 bucket
bucket_name = f"rag-data-{s3_suffix}"
try:
    s3.create_bucket(Bucket=bucket_name)
except botocore.exceptions.ClientError as e:
    error_code = e.response["Error"]["Code"]
    if error_code == "BucketAlreadyOwnedByYou":
        print(f"Bucket {bucket_name} already exists.")
    else:
        print(f"Error creating bucket: {e}")
        raise

# Create a vector store - OpenSearch Serverless index
## Step 1 - Create policies and collection

Setup a vector store and create an execution role. This is necessary for the bedrock agent to operate within AWS.

In [ ]:

vector_store_name = f'bedrock-sample-rag-{suffix}'
index_name = f"bedrock-sample-rag-index-{suffix}"
aoss_client = boto3_session.client('opensearchserverless')

# Create an execution role for bedrock knowledge bases
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']


In [ ]:

# Create security, network and data access policies within OSS
encryption_policy, network_policy, access_policy = create_policies_in_oss(
    vector_store_name=vector_store_name,
    aoss_client=aoss_client,
    bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn,
)

# Create a collection for vector search
collection = aoss_client.create_collection(name=vector_store_name, type="VECTORSEARCH")


In [ ]:
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'

This checks if the collection exists and waits for it to become active if it's still being created. It prints the collection's status, ensuring it's ready for use before moving on. The process involves:

1. **Checking if Active**: Confirms if the collection is ready.
2. **Waiting if Creating**: Waits and checks every 30 seconds if the collection is still being set up.
3. **Handling Non-existence**: Alerts if the collection doesn't exist.
4. **Reporting Other Statuses**: Notes any other collection status for troubleshooting.


In [ ]:
response = aoss_client.batch_get_collection(names=[vector_store_name])

# Initial check for collection existence and extraction of status
collection_exists = "collectionDetails" in response and response["collectionDetails"]
collection_status = collection_exists and response["collectionDetails"][0].get("status")

if collection_status == "ACTIVE":
    print("Collection already exists and is ACTIVE.")
elif collection_status == "CREATING":
    print("Collection is being created... Waiting for it to become ACTIVE.")
    while collection_status == "CREATING":
        time.sleep(30)  # Wait for 30 seconds
        response = aoss_client.batch_get_collection(names=[vector_store_name])
        collection_exists = (
            "collectionDetails" in response and response["collectionDetails"]
        )
        collection_status = collection_exists and response["collectionDetails"][0].get(
            "status", "CREATING"
        )  # Default to CREATING to continue loop if undefined
    print("Collection is now ACTIVE.")
elif not collection_exists:
    print("Collection details not found. The collection does not exist.")
else:
    print(f"Collection status: {collection_status}")

In [ ]:
# create oss policy and attach it to Bedrock execution role
create_oss_policy_attach_bedrock_execution_role(
    collection_id=collection_id, bedrock_kb_execution_role=bedrock_kb_execution_role
)

## Step 2 - Create vector index

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

index_name = f"bedrock-sample-index-{suffix}"
body_json = {
    "settings": {
        "index.knn": "true",
        "number_of_shards": 1,
        "knn.algo_param.ef_search": 512,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {
            "vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "name": "hnsw",
                    "engine": "faiss",
                    "space_type": "innerproduct",
                    "parameters": {"ef_construction": 512, "m": 16},
                },
            },
            "text": {"type": "text"},
            "text-metadata": {"type": "text"},
        }
    },
}
# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300,
)
# # It can take up to a minute for data access rules to be enforced
time.sleep(60)


In [ ]:
# Create index
response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
print('\nCreating index:')
print(response)
time.sleep(60) # index creation can take up to a minute

### Upoad data to S3 Bucket

In [ ]:
folder_path = 'knowledge_data/'  # Specify the folder path (with trailing '/')
s3_key = folder_path + 'hypertension.pdf'  # Combine folder path with file name

# Upload a CSV file to the S3 bucket
local_file_path = 'hypertension.pdf'

try:
    s3.upload_file(Filename=local_file_path, Bucket=bucket_name, Key=s3_key)
    print(f"File '{local_file_path}' uploaded to '{bucket_name}/{s3_key}'")
except Exception as e:
    print(f"Error uploading file to S3: {e}")
    raise

## Create Knowledge Base

Steps:

- Initialize OpenSearch serverless configuration which will include collection ARN, index name, vector field, text field, and metadata field.
- Initialize chunking strategy, based on which KB will split the documents into pieces of size equal to the chunk size mentioned in the chunkingStrategyConfiguration.
- Initialize the S3 configuration, which will be used to create the data source object later.
- Initialize the Titan embeddings model ARN, as this will be used to create the embeddings for each of the text chunks.


In [ ]:
opensearchServerlessConfiguration = {
    "collectionArn": collection["createCollectionDetail"]["arn"],
    "vectorIndexName": index_name,
    "fieldMapping": {
        "vectorField": "vector",
        "textField": "text",
        "metadataField": "text-metadata",
    },
}

chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {"maxTokens": 512, "overlapPercentage": 20},
}

s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
    # "inclusionPrefixes":["*.*"] # you can use this if you want to create a KB using data within s3 prefixes.
}

embeddingModelArn = (
    f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"
)

name = f"bedrock-sample-knowledge-base-{suffix}"
description = "Guideline for the pharmacological treatment of hypertension in adults"
roleArn = bedrock_kb_execution_role_arn


In [ ]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = name,
        description = description,
        roleArn = roleArn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [ ]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

In [ ]:
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

Create a data source, which will be associated with the knowledge base created above and ingest documents.

In [ ]:
# Create a DataSource in KnowledgeBase
create_ds_response = bedrock_agent_client.create_data_source(
    name=name,
    description=description,
    knowledgeBaseId=kb["knowledgeBaseId"],
    dataSourceConfiguration={"type": "S3", "s3Configuration": s3Configuration},
    vectorIngestionConfiguration={
        "chunkingConfiguration": chunkingStrategyConfiguration
    },
)
ds = create_ds_response["dataSource"]


In [ ]:
# Get DataSource 
bedrock_agent_client.get_data_source(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

### Start ingestion job

Once the KB and data source is created, we can start the ingestion job. 

In [ ]:
# Start an ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [ ]:
job = start_job_response["ingestionJob"]

In [ ]:
# Get job
while job["status"] != "COMPLETE":
    get_job_response = bedrock_agent_client.get_ingestion_job(
        knowledgeBaseId=kb["knowledgeBaseId"],
        dataSourceId=ds["dataSourceId"],
        ingestionJobId=job["ingestionJobId"],
    )
    job = get_job_response["ingestionJob"]
time.sleep(40)

In [ ]:
kb_id = kb["knowledgeBaseId"] # After creation, you can just replace this value with your kb id to avoid running cells again

In [ ]:
%store kb_id

# Create Patient Database
## Step 1 - Create and Upload Files to S3

First take the dataset and create a new csv file with your (the doctor's) email on it 

In [ ]:
import pandas as pd

# Read CSV file into pandas DataFrame
input_file = 'input.csv'
df = pd.read_csv(input_file)

# Add 'email' column with specified email address
email_to_add = 'example@example.com'
df['email'] = email_to_add

# Write DataFrame to CSV file
output_file = 'output.csv'
df.to_csv(output_file, index=False)


Create a new S3 bucket for the patient data and upload it

In [ ]:
# Create an S3 client
s3 = boto3.client('s3')

# Create an S3 bucket
patient_bucket_name = f'patient-data-{s3_suffix}'

try:
    s3.create_bucket(Bucket=patient_bucket_name)
except botocore.exceptions.ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == 'BucketAlreadyOwnedByYou':
        print(f"Bucket {bucket_name} already exists.")
    else:
        print(f"Error creating bucket: {e}")
        raise

In [ ]:
# Upload a CSV file with ptient data to the S3 bucket
local_file_path_2 = output_file
s3_key_dataset = 'patient_data_w_email.csv"

try:
    s3.upload_file(Filename=local_file_path_2, Bucket=patient_bucket_name, Key=s3_key_dataset)
except ClientError as e:
    print(f"Error uploading file to S3: {e}")
    raise

## Step 2 - Create DynamoDB table

Create a new DynamoDB table by importing the csv from S3. You might have to wait a bit before the table is created and you can use it in the next steps

In [ ]:
# Create a DynamoDB client
dynamodb = boto3.client('dynamodb')

table_name = "PatientInfo" # change to your table name  

# Create the DynamoDB table using the import_table client
response = dynamodb.import_table(
    S3BucketSource={
        'S3Bucket': patient_bucket_name,
        'S3KeyPrefix': s3_key_dataset
    },
    InputFormat='CSV',
    TableCreationParameters={
        'TableName': table_name, 
        'AttributeDefinitions': [
            {
                'AttributeName': 'Patient ID',
                'AttributeType': 'S'
            },
        ],
        'KeySchema': [
            {
                'AttributeName': 'Patient ID',
                'KeyType': 'HASH'
            },
        ],
        'BillingMode': 'PAY_PER_REQUEST'
    }
)

Waiting on the DynamoDB import and table creation to be complete, checking the status. This cell was added because the import from S3 doesn't create a table immediately, so it's not recognized as soon as you run the cell 

In [ ]:
print(f"Initiating table import for '{table_name}'. Waiting for availability...")
max_attempts = 60
last_status = None
for attempt in range(max_attempts):
    try:
        table_desc = dynamodb.describe_table(TableName=table_name)
        current_status = table_desc['Table']['TableStatus']
        if current_status != last_status:
            print(f"Table '{table_name}' status: {current_status}.")
            last_status = current_status
        if current_status == 'ACTIVE':
            print(f"Table '{table_name}' is now ACTIVE and ready for use.")
            break
        else:
            time.sleep(5)  # Wait for 5 seconds before checking again
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            if last_status is None:
                print("Table not found. Waiting for it to become available...")
                last_status = 'NOT FOUND'
            time.sleep(5)
        else:
            raise
else:
    print(f"Table '{table_name}' did not become ACTIVE after {max_attempts * 5} seconds.")


# Generate and Send Recommendations
## Step 1 - Retrive info and promp the model

We are now moving to the point to choose our patient and retrieve the patient's information from the database 

In [ ]:
# Choose the patient based on ID

patient_id = '2' # Change this to any patient ID in the database 

In [ ]:
try:
    # Retrieve item from DynamoDB
    response = dynamodb.get_item(
        TableName=table_name, Key={"Patient ID": {"S": patient_id}}
    )

    # Extract patient information if item exists
    item = response.get("Item")
    if item:
        patient_info = {key: value.get("S") for key, value in item.items()}
        print("Patient Information:", patient_info)
    else:
        print("Patient not found")
        patient_info = None

except Exception as e:
    print("Error retrieving patient information:", e)
    patient_info = None


### Get Documents from Knowledge Base

Create a AmazonKnowledgeBasesRetriever object from LangChain which will call the Retreive API provided by Knowledge Bases for Amazon Bedrock which converts user queries into embeddings, searches the knowledge base, and returns the relevant results.

In [ ]:
# Simulate getting recommendation using BedrockChat
# Create the LangChain components
from langchain.prompts import PromptTemplate
import pprint as pp

bedrock_client = boto3.client("bedrock-runtime")

model_id = "anthropic.claude-3-haiku-20240307-v1:0"
llm = BedrockChat(model_id=model_id, client=bedrock_client)

question = (
    f"What recommendations would you give to the doctor based on the {patient_info}?"
)
retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id=kb_id,
    retrieval_config={
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            "overrideSearchType": "SEMANTIC",  # optional
        }
    },
)

docs = retriever.get_relevant_documents(query=question)
pp.pprint(docs)


### Prompt the Model 
Here we are prompting the model giving it context, patient info, and guidelines

In [ ]:
PROMPT_TEMPLATE = """
Human: You are a medical assistant tasked with writing email recommendations to a doctor who has assisted a patient. 
You should keep a formal and professional language. 
You should present yourself as Dr. Claude, the patient's AI medical assistant, and provide next steps for the patient.
You should use medical technical terms but should not refer specific studies in your answer.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
Correlate the patient's information with the medical info from the context to provide a medically accurate response. 

<context>
{context}
</context>

<question>
{question}
</question>

Never say "based on the context or information provided". It should come across that you already know this knowledge beforehand.
Finish the email with the source of your recommendations.

Assistant:"""

claude_prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)


Integrating the retriever and the LLM defined above with RetrievalQA Chain.

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": claude_prompt}
)

In [ ]:
answer = qa.invoke(question)

recommendation = answer['result']

print(recommendation)

## Step 2 - Create SNS Topic and Send email

Creating the SNS Topic 

In [ ]:
# Create an SNS client
sns = boto3.client('sns')

# Create an SNS topic
topic_name = "DrClaude"
response = sns.create_topic(Name=topic_name)
topic_arn = response['TopicArn']
print(f"Created SNS topic: {topic_name}")

Subscribing to the topic

In [ ]:
# Subscribe an email to the SNS topic
email_address = "example@example.com" # put your own email here 

subscription_response = sns.subscribe(
    TopicArn=topic_arn,
    Protocol='email',
    Endpoint=email_address
    Attributes={
        "FilterPolicy": '{"email": ["' + email_address + '"]}' #When you have multiple doctor emails, the email is only sent to the patient's doctor
    }
)

print(f"Subscribed email {email_address} to topic {topic_name}\n")
print("Check your inbox to confirm the subscription!")

Checking if doctor's email is subscribed to the topic and sending it upon confirmation

In [ ]:
# Check if the patient's email is subscribed to the SNS topic
response = sns.list_subscriptions_by_topic(TopicArn=topic_arn)

# Extract the list of subscribed emails
subscribed_emails = [
    subscription["Endpoint"] for subscription in response["Subscriptions"]
]

doctor_email = patient_info["email"]

# If the patient's email is subscribed, send the email
if doctor_email in subscribed_emails:
    # Set the email subject and message
    subject = "Dr. Claude's Medical Recommendations"
    message = f"{recommendation}"

    # Send the email using SNS
    response = sns.publish(
        TopicArn=topic_arn,
        Subject=subject,
        Message=message,
        MessageAttributes={
            "email": {"DataType": "String", "StringValue": doctor_email}
        },
    )
    print("Email sent successfully.")
    print("Response:", response)
else:
    print(f"The patient's email ({doctor_email}) is not subscribed to the SNS topic.")


# Clean up

### Delete KnowledgeBase

In [ ]:
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)

In [ ]:
bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=kb['knowledgeBaseId'])
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb['knowledgeBaseId'])
oss_client.indices.delete(index=index_name)
aoss_client.delete_collection(id=collection_id)
aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

### Delete role and policies

In [ ]:
from policy_creator import delete_iam_role_and_policies
delete_iam_role_and_policies()

### Delete RAG data S3 objects

In [ ]:
s3 = boto3.resource('s3')
bucket_1 = s3.Bucket(bucket_name)

# Delete all objects in the bucket before deleting it
bucket_1.objects.all().delete()

# Delete the bucket
bucket_1.delete()

### Delete patient data S3 objects

In [ ]:
bucket_2 = s3.Bucket(patient_bucket_name)

# Delete all objects in the bucket before deleting it
bucket_2.objects.all().delete()

# Delete the bucket
bucket_2.delete()

### Delete DynamoDB Table

In [ ]:
dynamodb.delete_table(TableName=table_name)

### Delete SNS Topic

In [ ]:
sns.delete_topic(TopicArn=topic_arn)